# compas_fea2 simple script approach

This example covers the analysis of a simple 2D frame structure generated completly from scratch. 

In [1]:
from compas_fea2.backends.abaqus.model import Model
from compas_fea2.backends.abaqus.model import Part
from compas_fea2.backends.abaqus.model import Node
from compas_fea2.backends.abaqus.model import ElasticIsotropic
from compas_fea2.backends.abaqus.model import BoxSection
from compas_fea2.backends.abaqus.model import BeamElement
from compas_fea2.backends.abaqus.model import Set

from compas_fea2.backends.abaqus.problem import Problem
from compas_fea2.backends.abaqus.problem import FixedDisplacement
from compas_fea2.backends.abaqus.problem import RollerDisplacementXZ
from compas_fea2.backends.abaqus.problem import PointLoad
from compas_fea2.backends.abaqus.problem import FieldOutput
from compas_fea2.backends.abaqus.problem import GeneralStaticStep

## Model generation

The fist step of an FEA is the generation of the analysis `model`. 
A model is always composed by:

1) geometry: nodes, connectivity among nodes, parts in which nodes and elements are groups, etc.
The geometry can either be created from scratch (as in this example) or be imported from other datastructures (networks, meshes, etc).

2) properties:  materials, sections, elements, interactions, etc. Properties are applied to the geometry to simulate the mechanical behaviour. 

In a compas workflow, geometry and properties are provided by the compas_package that creates the structure to analyse. 


In [2]:
model = Model(name='structural_model')

print(model)


compas_fea2 Model object
------------------------
name            : structural_model
# of parts      : 0
# of instances  : 0


### Parts

`Parts` are subregions of the model that can be considered to be independent from each other. However similar parts of a model can be associated to different `Parts`. 

A good example of a `part` can be a brick 

In [3]:
model.add_part(Part(name='part-1'))

print(model.parts['part-1'])


compas_fea2 Part object
-----------------------
name            : part-1
# of nodes      : 0
# of elements   : 0


Nodes can be added to the `Part` or directly to the model specifying the name of the `Part` to which they belong.

In [4]:
for x in range(0, 1100, 100):
    model.add_node(Node(xyz=[x, 0.0, 0.0]), part='part-1')
for y in range(100, 600, 100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')
for x in range(900, -100, -100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')
for y in range(400, 0, -100):
    model.add_node(Node(xyz=[x, y, 0.0]), part='part-1')

print(model.parts['part-1'].nodes[5])


compas_fea2 Node object
-----------------------
label      : n-5
key        : 5
x          : 500.0
y          : 0.0
z          : 0.0
ex         : None
ey         : None
ez         : None
mass       : None


In [5]:
# Define materials
model.add_material(ElasticIsotropic(name='mat_A', E=29000, v=0.17, p=2.5e-9))
model.add_material(ElasticIsotropic(name='mat_B', E=25000, v=0.17, p=2.5e-9))

print(model.materials['mat_A'])


compas_fea2 ElasticIsotropic object
-----------------------------------
name        : mat_A
E           : {'E': 29000}
v           : {'v': 0.17}
G           : {'G': 12393.162393162394}
p           : 2.5e-09


In [6]:
# Define sections
model.add_section(BoxSection(name='section_A', material='mat_A', a=20, b=80, t1=5, t2=5, t3=5, t4=5))
model.add_section(BoxSection(name='section_B', material='mat_B', a=20, b=80, t1=5, t2=5, t3=5, t4=5))

print(model.sections['section_A'])


compas_fea2 Section object
--------------------------
name  : section_A


In [7]:
# Generate elements between nodes
elements = []
for e in range(len(model.parts['part-1'].nodes)-1):
    elements.append((BeamElement(connectivity=[e, e+1], section='section_A')))
model.add_elements(elements=elements, part='part-1')
model.add_element(element=BeamElement(connectivity=[29, 0], section='section_B'), part='part-1')

print(model.parts['part-1'].elements[15])



compas_fea2 BeamElement object
------------------------------
key        : 15
etype      : beam
connectivity : [15, 16]


In [8]:
# Define sets for boundary conditions and loads
model.add_assembly_set(Set(name='fixed', selection=[0], stype='nset'), instance='part-1-1')
model.add_assembly_set(Set(name='roller', selection=[10], stype='nset'), instance='part-1-1')
model.add_assembly_set(Set(name='pload', selection=[20], stype='nset'), instance='part-1-1')

## Problem definition

In [9]:

# Create the Problem object
problem = Problem(name='test_structure', model=model)

# Assign boundary conditions to the node stes
problem.add_bcs(bcs=[RollerDisplacementXZ(name='bc_roller', bset='roller'),
                        FixedDisplacement(name='bc_fix', bset='fixed')])

# Assign a point load to the node set
problem.add_load(load=PointLoad(name='pload', lset='pload', y=-1000))

# Define the field outputs required
problem.add_field_output(fout=FieldOutput(name='fout'))

# Define the analysis step
problem.add_step(step=GeneralStaticStep(name='gstep', loads=['pload'], field_output=['fout']))

# Solve the problem
# my_structure.write_input_file(path='C:/temp/test_structure')
problem.analyse(path='C:/temp/test_structure')


***** Input file generated: C:/temp/test_structure/test_structure.inp *****

b'**********************************************************************'
b'** Visual Studio 2019 Developer Command Prompt v16.6.2'
b'** Copyright (c) 2020 Microsoft Corporation'
b'**********************************************************************'
b"[vcvarsall.bat] Environment initialized for: 'x64'"
b''
b'Intel(R) MPI Library 2019 Update 7 for Windows* Target Build Environment for Intel(R) 64 applications'
b'Copyright 2007-2020 Intel Corporation.'
b''
b'Copyright (C) 1985-2019 Intel Corporation. All rights reserved.'
b'Intel(R) Compiler 19.1 Update 1 (package 216)'
b''
b'**********************************************************************'
b'** Visual Studio 2019 Developer Command Prompt v16.6.2'
b'** Copyright (c) 2020 Microsoft Corporation'
b'**********************************************************************'
b"[vcvarsall.bat] Environment initialized for: 'x64'"
b'Analysis initiated from SIMULIA 